## Decision Trees in Practice

In this assignment we will explore various techniques for preventing overfitting in decision trees. We will extend the implementation of the binary decision trees that we implemented in the previous assignment. You will have to use your solutions from this previous assignment and extend them.

In this assignment you will:

* Implement binary decision trees with different early stopping methods.<br>
* Compare models with different stopping parameters.<br>
* Visualize the concept of overfitting in decision trees.

In [1]:
import numpy as np
import pandas as pd

1. Load in the LendingClub dataset with the software of your choice.

In [2]:
loans = pd.read_csv('lending-club-data.csv')

c:\users\rituraj\virtual-env\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (19,47) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
loans.shape

(122607, 68)

In [4]:
loans.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,sub_grade_num,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,...,0.4,1.0,1.0,1.0,0,8.14350,20141201T000000,1,1,1
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,0.8,1.0,1.0,1.0,1,2.39320,20161201T000000,1,1,1
2,1077175,1313524,2400,2400,2400,36 months,15.96,84.33,C,C5,...,1.0,1.0,1.0,1.0,0,8.25955,20141201T000000,1,1,1
3,1076863,1277178,10000,10000,10000,36 months,13.49,339.31,C,C1,...,0.2,1.0,1.0,1.0,0,8.27585,20141201T000000,0,1,1
4,1075269,1311441,5000,5000,5000,36 months,7.90,156.46,A,A4,...,0.8,1.0,1.0,1.0,0,5.21533,20141201T000000,1,1,1


2. Reassign the labels to have +1 for a safe loan, and -1 for a risky (bad) loan. You should have code analogous to

In [5]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x: 1 if x == 0  else -1)
loans.drop('bad_loans', axis = 'columns',  inplace = True)

3. We will only be considering these four features:

In [6]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'

In [7]:
loans_data = loans[features+[target]]
loans_data.head()

,grade,term,home_ownership,emp_length,safe_loans
0,B,36 months,RENT,10+ years,1
1,C,60 months,RENT,< 1 year,-1
2,C,36 months,RENT,10+ years,1
3,C,36 months,RENT,10+ years,1
4,A,36 months,RENT,3 years,1


### Subsample dataset to make sure classes are balanced

4. Just as we did in the previous assignment, we will undersample the larger class (safe loans) in order to balance out our dataset. This means we are throwing away many data points. You should have code analogous to

In [8]:
# safe_loans_raw = loans[loans[target] == 1]
# risky_loans_raw = loans[loans[target] == -1]

# # Since there are less risky loans than safe loans, find the ratio of the sizes
# # and use that percentage to undersample the safe loans.
# percentage = len(risky_loans_raw)/float(len(safe_loans_raw))
# safe_loans = safe_loans_raw.sample(percentage, seed = 1)
# risky_loans = risky_loans_raw
# loans_data = risky_loans.append(safe_loans)

# print "Percentage of safe loans                 :", len(safe_loans) / float(len(loans_data))
# print "Percentage of risky loans                :", len(risky_loans) / float(len(loans_data))
# print "Total number of loans in our new dataset :", len(loans_data)

### Train-validation split

In [9]:
import json

In [10]:
with open('module-6-assignment-train-idx.json') as file:
    train_idx = json.load(file)

In [11]:
with open('module-6-assignment-validation-idx.json') as file:
    validation_idx = json.load(file)

In [12]:
train = loans_data.iloc[train_idx]

In [13]:
validation = loans_data.iloc[validation_idx]

In [14]:
train.shape, validation.shape

((37224, 5), (9284, 5))

In [15]:
# appending train and validation data for data preprocessing
total_data = train.append(validation)

In [16]:
total_data['emp_length'].unique()

array(['< 1 year', '4 years', '3 years', '10+ years', '1 year', '9 years',
       '8 years', '7 years', '5 years', '2 years', nan, '6 years'],
      dtype=object)

In [17]:
total_data['emp_length'] = total_data['emp_length'].apply(lambda text: str(text).replace(' ', ''))

In [18]:
total_data['emp_length'].unique()

array(['<1year', '4years', '3years', '10+years', '1year', '9years',
       '8years', '7years', '5years', '2years', 'nan', '6years'],
      dtype=object)

In [19]:
total_data.reset_index(inplace = True)

In [20]:
total_data.drop('index', axis = 1, inplace = True)
total_data.head()

,grade,term,home_ownership,emp_length,safe_loans
0,C,60 months,RENT,<1year,-1
1,F,60 months,OWN,4years,-1
2,B,60 months,RENT,<1year,-1
3,C,36 months,RENT,<1year,-1
4,B,36 months,RENT,3years,-1


## Transform categorical data into binary features

In this assignment, we will implement **binary decision trees** (decision trees for binary features, a specific case of categorical variables taking on two values, e.g., true/false). Since all of our features are currently categorical features, we want to turn them into binary features.


In [21]:
categorical_features = list(total_data.select_dtypes('object').columns)
categorical_features

['grade', 'term', 'home_ownership', 'emp_length']

In [22]:
# for feature in categorical_features:
#     dummy = pd.get_dummies(total_data[feature], prefix = feature, dummy_na = True)
#     total_data = total_data.join(dummy)
#     total_data.drop(feature, axis = 1, inplace = True)

In [23]:
from sklearn.preprocessing import LabelEncoder

In [24]:
from sklearn.preprocessing import OneHotEncoder

In [25]:
encoder = OneHotEncoder()

In [26]:
for feature in categorical_features:
    encoder.fit(total_data[[feature]])
    name_of_new_features = encoder.categories_[0]
    enc_df = pd.DataFrame()
    enc_df = pd.DataFrame(encoder.transform(total_data[[feature]]).toarray())
    fetures_dict = {}
    for i in range(len(name_of_new_features)):
        fetures_dict[enc_df.columns[i]] =  str(feature) +'_'+ str(name_of_new_features[i])
    enc_df = enc_df.rename(columns = fetures_dict)
    total_data = total_data.join(enc_df)
    total_data.drop(feature, axis = 1, inplace = True)

In [27]:
total_data.head()

,safe_loans,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,term_ 36 months,term_ 60 months,...,emp_length_2years,emp_length_3years,emp_length_4years,emp_length_5years,emp_length_6years,emp_length_7years,emp_length_8years,emp_length_9years,emp_length_<1year,emp_length_nan
0,-1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


5. This technique of turning categorical variables into binary variables is called one-hot encoding. Using the software of your choice, perform one-hot encoding on the four features described above. **You should now have 25 binary features.**

In [28]:
total_data.shape

(46508, 26)

In [29]:
total_data.columns

Index(['safe_loans', 'grade_A', 'grade_B', 'grade_C', 'grade_D', 'grade_E',
       'grade_F', 'grade_G', 'term_ 36 months', 'term_ 60 months',
       'home_ownership_MORTGAGE', 'home_ownership_OTHER', 'home_ownership_OWN',
       'home_ownership_RENT', 'emp_length_10+years', 'emp_length_1year',
       'emp_length_2years', 'emp_length_3years', 'emp_length_4years',
       'emp_length_5years', 'emp_length_6years', 'emp_length_7years',
       'emp_length_8years', 'emp_length_9years', 'emp_length_<1year',
       'emp_length_nan'],
      dtype='object')

### Train-validation split

6. Split the data into a train-validation. Call these datasets train_data and validation_data,

In [30]:
train_data = total_data[:len(train_idx)]

In [31]:
train_data

,safe_loans,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,term_ 36 months,term_ 60 months,...,emp_length_2years,emp_length_3years,emp_length_4years,emp_length_5years,emp_length_6years,emp_length_7years,emp_length_8years,emp_length_9years,emp_length_<1year,emp_length_nan
0,-1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37219,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
37220,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
37221,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37222,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
validation_data = total_data[len(train_idx):].reset_index().drop('index', axis = 1)

In [33]:
validation_data

,safe_loans,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,term_ 36 months,term_ 60 months,...,emp_length_2years,emp_length_3years,emp_length_4years,emp_length_5years,emp_length_6years,emp_length_7years,emp_length_8years,emp_length_9years,emp_length_<1year,emp_length_nan
0,-1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9279,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9280,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9281,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9282,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [34]:
new_features  = list(total_data.columns)
new_features.remove('safe_loans')
print(new_features)

['grade_A', 'grade_B', 'grade_C', 'grade_D', 'grade_E', 'grade_F', 'grade_G', 'term_ 36 months', 'term_ 60 months', 'home_ownership_MORTGAGE', 'home_ownership_OTHER', 'home_ownership_OWN', 'home_ownership_RENT', 'emp_length_10+years', 'emp_length_1year', 'emp_length_2years', 'emp_length_3years', 'emp_length_4years', 'emp_length_5years', 'emp_length_6years', 'emp_length_7years', 'emp_length_8years', 'emp_length_9years', 'emp_length_<1year', 'emp_length_nan']


## Early stopping methods for decision trees

In this section, we will extend the binary tree implementation from the previous assignment in order to handle some early stopping conditions. Recall the 3 early stopping methods that were discussed in lecture:

1. Reached a maximum depth. (set by parameter max_depth).
2. Reached a minimum node size. (set by parameter min_node_size).
3. Don't split if the gain in error reduction is too small. (set by parameter min_error_reduction).

For the rest of this assignment, we will refer to these three as early stopping conditions 1, 2, and 3.

#### Early stopping condition 1: Maximum depth
Recall that we already implemented the maximum depth stopping condition in the previous assignment. In this assignment, we will experiment with this condition a bit more and also write code to implement the 2nd and 3rd early stopping conditions.

We will be reusing code from the previous assignment and then building upon this. We will alert you when you reach a function that was part of the previous assignment so that you can simply copy and past your previous code.

#### Early stopping condition 2: Minimum node size
The function reached_minimum_node_size takes 2 arguments:

    1. The data (from a node)
    2. The minimum number of data points that a node is allowed to split on, min_node_size.

7. This function simply calculates whether the number of data points at a given node is less than or equal to the specified minimum node size. This function will be used to detect this early stopping condition in the decision_tree_create function. Your code should be analogous to

In [35]:
def reached_minimum_node_size(data, min_node_size):
    # Return True if the number of data points is less than or equal to the minimum node size.
    ## YOUR CODE HERE
    if len(data)<= min_node_size:
        return True
    else:
        return False

1. **Quiz question:** Given an intermediate node with 6 safe loans and 3 risky loans, if the min_node_size parameter is 10, what should the tree learning algorithm do next?<br>
**Ans: -** Create a leaf and return it

#### Early stopping condition 3: Minimum gain in error reduction
The function error_reduction takes 2 arguments:

    1. The error before a split, error_before_split.
    2. The error after a split, error_after_split.

8. This function computes the gain in error reduction, i.e., the difference between the error before the split and that after the split. This function will be used to detect this early stopping condition in the decision_tree_create function. Your code should be analogous to

In [36]:
def error_reduction(error_before_split, error_after_split):
    # Return the error before the split minus the error after the split.
    ## YOUR CODE HERE
    return error_before_split - error_after_split

2. **Quiz question:** Assume an intermediate node has 6 safe loans and 3 risky loans. For each of 4 possible features to split on, the error reduction is 0.0, 0.05, 0.1, and 0.14, respectively. If the minimum gain in error reduction parameter is set to 0.2, what should the tree learning algorithm do next?<br>
**Ans:** Create a leaf and return it

## Decision tree implementation

In this section, we will implement binary decision trees from scratch. There are several steps involved in building a decision tree. For that reason, we have split the entire assignment into several sections.

#### Function to count number of mistakes while predicting majority class

 Now, we will write a function that calculates the number of misclassified examples when predicting the majority class. This will be used to help determine which feature is the best to split on at a given node of the tree.

**Note:** Keep in mind that in order to compute the number of mistakes for a majority classifier, we only need the label (y values) of the data points in the node.

`Steps to follow:`

* Step 1: Calculate the number of safe loans and risky loans.
* Step 2: Since we are assuming majority class prediction, all the data points that are not in the majority class are considered mistakes.
* Step 3: Return the number of mistakes.

9. Recall from the previous assignment that we wrote a function intermediate_node_num_mistakes that calculates the number of misclassified examples when predicting the majority class. This is used to help determine which feature is best to split on at a given node of the tree. Use your code from the previous assignment

In [37]:
def intermediate_node_num_mistakes(labels_in_node):
    # Corner case: If labels_in_node is empty, return 0
    if len(labels_in_node) == 0:
        return 0    
    # Count the number of 1's (safe loans)
    ## YOUR CODE HERE
    count_one = len(labels_in_node[labels_in_node==1])
    # Count the number of -1's (risky loans)
    ## YOUR CODE HERE
    count_minus_one = (labels_in_node == -1).sum()
    # Return the number of mistakes that the majority classifier makes.
    ## YOUR CODE HERE
    return min(count_one, count_minus_one)


### Function to pick best feature to split on

10. We then wrote a function best_splitting_feature that finds the best feature to split on given the data and a list of features to consider. Use your code from the previous assignment. It should be analogous to

The function best_splitting_feature takes 3 arguments:

1. The data
2. The features to consider for splits (a list of strings of column names to consider for splits)
3. The name of the target/label column (string)

Follow these steps to implement best_splitting_feature:

* Step 1: Loop over each feature in the feature list
* Step 2: Within the loop, split the data into two groups: one group where all of the data has feature value 0 or False (we will call this the left split), and one group where all of the data has feature value 1 or True (we will call this the right split). Make sure the left split corresponds with 0 and the right split corresponds with 1 to ensure your implementation fits with our implementation of the tree building process.
* Step 3: Calculate the number of misclassified examples in both groups of data and use the above formula to compute theclassification error.
* Step 4: If the computed error is smaller than the best error found so far, store this feature and its error.

In [38]:
def best_splitting_feature(data, features, target):
    
    target_values = data[target]
    best_feature = None # Keep track of the best feature 
    best_error = 10     # Keep track of the best error so far 
    # Note: Since error is always <= 1, we should intialize it with something larger than 1.

    # Convert to float to make sure error gets computed correctly.
    num_data_points = float(len(data))  
    
    # Loop through each feature to consider splitting on that feature
    for feature in features:
        
        # The left split will have all data points where the feature value is 0
        left_split = data[data[feature] == 0]
        
        # The right split will have all data points where the feature value is 1
        ## YOUR CODE HERE
        right_split =  data[data[feature]==1]
            
        # Calculate the number of misclassified examples in the left split.
        # Remember that we implemented a function for this! (It was called intermediate_node_num_mistakes)
        # YOUR CODE HERE
        left_mistakes = intermediate_node_num_mistakes(left_split[target])

        # Calculate the number of misclassified examples in the right split.
        ## YOUR CODE HERE
        right_mistakes = intermediate_node_num_mistakes(right_split[target])
            
        # Compute the classification error of this split.
        # Error = (# of mistakes (left) + # of mistakes (right)) / (# of data points)
        ## YOUR CODE HERE
        error = (left_mistakes+right_mistakes)/num_data_points

        # If this is the best error we have found so far, store the feature as best_feature and the error as best_error
        ## YOUR CODE HERE
        if error < best_error:
            best_feature = feature
            best_error = error
    
    return best_feature # Return the best feature we found

In [39]:
# { 
#    'is_leaf'            : True/False.
#    'prediction'         : Prediction at the leaf node.
#    'left'               : (dictionary corresponding to the left tree).
#    'right'              : (dictionary corresponding to the right tree).
#    'splitting_feature'  : The feature that this node splits on
# }


11. Finally, recall the function create_leaf from the previous assignment, which creates a leaf node given a set of target values. Use your code from the previous assignment

In [40]:
def create_leaf(target_values):    
    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True   }   ## YOUR CODE HERE 
   
    # Count the number of data points that are +1 and -1 in this node.
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])    

    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] = 1         ## YOUR CODE HERE
    else:
        leaf['prediction'] = -1        ## YOUR CODE HERE        

    # Return the leaf node
    return leaf 

## Incorporating new early stopping conditions in binary decision tree implementation

We have provided a function that learns the decision tree recursively and implements 3 stopping conditions:(From previous assignment)

1. **Stopping condition 1:** All data points in a node are from the same class.
2. **Stopping condition 2:** No more features to split on.
3. **Additional stopping condition:** In addition to the above two stopping conditions covered in lecture, in this assignment we will also consider a stopping condition based on the **max_depth** of the tree. By not letting the tree grow too deep, we will save computational effort in the learning process.

12. Now, you will implement a function that builds a decision *tree handling the three early stopping conditions* described in this assignment. In particular, you will write code to detect early stopping conditions 2 and 3. You implemented above the functions needed to detect these conditions. The 1st early stopping condition, **max_depth**, was implemented in the previous assignment and you will not need to reimplement this. In addition to these early stopping conditions, the typical stopping conditions of having no mistakes or no more features to split on (which we denote by "stopping conditions" 1 and 2) are also included as in the previous assignment.

**Implementing early stopping condition 2: minimum node size:**

* **Step 1:** Use the function reached_minimum_node_size that you implemented earlier to write an if condition to detect whether we have hit the base case, i.e., the node does not have enough data points and should be turned into a leaf. Don't forget to use the min_node_size argument.
* **Step 2:** Return a leaf. This line of code should be the same as the other (pre-implemented) stopping conditions.

**Implementing early stopping condition 3: minimum error reduction:**

**Note:** This has to come after finding the best splitting feature so we can calculate the error after splitting in order to calculate the error reduction. Recall that classification error is defined as:

   **classification error = #mistakes / #total examples**

* Step 1: Calculate the classification error before splitting. 
* Step 2: Calculate the classification error after splitting. This requires calculating the number of mistakes in the left and right splits, and then dividing by the total number of examples.
* Step 3: Use the function error_reduction to that you implemented earlier to write an if condition to detect whether the reduction in error is less than the constant provided (min_error_reduction). Don't forget to use that argument.
* Step 4: Return a leaf. This line of code should be the same as the other (pre-implemented) stopping conditions.

In [41]:
def decision_tree_create(data, features, target, current_depth = 0, 
                         max_depth = 10, min_node_size=1, 
                         min_error_reduction=0.0):
    
    remaining_features = features[:] # Make a copy of the features.
    
    target_values = data[target]
    print("--------------------------------------------------------------------")
    print("Subtree, depth = %s (%s data points)." % (current_depth, len(target_values)))
    
    
    # Stopping condition 1: All nodes are of the same type.
    if intermediate_node_num_mistakes(target_values) == 0:
        print("Stopping condition 1 reached. All data points have the same target value.")                
        return create_leaf(target_values)
    
    # Stopping condition 2: No more features to split on.
    if remaining_features == []:
        print("Stopping condition 2 reached. No remaining features.")                
        return create_leaf(target_values)    
    
    # Early stopping condition 1: Reached max depth limit.
    if current_depth >= max_depth:
        print("Early stopping condition 1 reached. Reached maximum depth.")
        return create_leaf(target_values)
    
    # Early stopping condition 2: Reached the minimum node size.
    # If the number of data points is less than or equal to the minimum size, return a leaf.
    if reached_minimum_node_size(data, min_node_size) == True:  ## YOUR CODE HERE 
        print("Early stopping condition 2 reached. Reached minimum node size.")
        return  create_leaf(target_values) ## YOUR CODE HERE
    
    # Find the best splitting feature
    splitting_feature = best_splitting_feature(data, features, target)
    
    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    
    # Early stopping condition 3: Minimum error reduction
    # Calculate the error before splitting (number of misclassified examples 
    # divided by the total number of examples)
    error_before_split = intermediate_node_num_mistakes(target_values) / float(len(data))
    
    # Calculate the error after splitting (number of misclassified examples 
    # in both groups divided by the total number of examples)
    left_mistakes =  intermediate_node_num_mistakes(left_split[target])  ## YOUR CODE HERE
    right_mistakes = intermediate_node_num_mistakes(right_split[target])  ## YOUR CODE HERE
    error_after_split = (left_mistakes + right_mistakes) / float(len(data))
    
    # If the error reduction is LESS THAN OR EQUAL TO min_error_reduction, return a leaf.
    if error_reduction(error_before_split, error_after_split) <=  min_error_reduction:  ## YOUR CODE HERE
        print("Early stopping condition 3 reached. Minimum error reduction.")
        return  create_leaf(target_values)  ## YOUR CODE HERE 
    
    
    remaining_features.remove(splitting_feature)
    print("Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split)))
    
    
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, 
                                     current_depth + 1, max_depth, min_node_size, min_error_reduction)        
    
    ## YOUR CODE HERE
    right_tree = decision_tree_create(right_split, remaining_features, target, 
                                     current_depth + 1, max_depth, min_node_size, min_error_reduction)   
    
    
    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

### Build the tree!

13. Now that your code is working, train a tree model on the train_data with

* max_depth = 6
* min_node_size = 100,
* min_error_reduction = 0.0

**Warning:** This code block may take a minute to learn. Call this model my_decision_tree_new

In [42]:
my_decision_tree_new = decision_tree_create(train_data, new_features, 'safe_loans', max_depth = 6, 
                                min_node_size = 100, min_error_reduction=0.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Early stopping condition 3 reached. Minimum error reduction.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Split on feature emp_length_nan. (96, 5)
--------------------------------------------------------------------
Subtree, depth = 3 (96 data points).
Early stopping condition 2 reached. Reached minimum node size.
--------------------------------------------------------------------
Subtree, depth = 3 (5 data points).
Early stopping condition 2 reached. Reached minimum node size.
-------------------------------------------

In [43]:
my_decision_tree_new['left']['right']

{'is_leaf': False,
 'prediction': None,
 'splitting_feature': 'emp_length_nan',
 'left': {'splitting_feature': None,
  'left': None,
  'right': None,
  'is_leaf': True,
  'prediction': 1},
 'right': {'splitting_feature': None,
  'left': None,
  'right': None,
  'is_leaf': True,
  'prediction': -1}}

14. Let's now train a tree model ignoring early stopping conditions 2 and 3 so that we get the same tree as in the previous assignment. To ignore these conditions, we set min_node_size=0 and min_error_reduction=-1 (a negative value). Call this model my_decision_tree_old. Your code should be analogous to

In [44]:
my_decision_tree_old = decision_tree_create(train_data, new_features, 'safe_loans', max_depth = 6, min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
E

Split on feature grade_G. (20638, 96)
--------------------------------------------------------------------
Subtree, depth = 6 (20638 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 6 (96 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 5 (932 data points).
Split on feature grade_A. (702, 230)
--------------------------------------------------------------------
Subtree, depth = 6 (702 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 6 (230 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 4 (358 data points).
Split on feature emp_length_8years. (347, 11)


### Making predictions with a decision tree

15. Recall that in the previous assignment you implemented a function classify to classify a new point x using a given tree. We will need that function here. Use your code from the previous assignment. 

In [45]:
def classify(tree, x, annotate = False):   
    # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate: 
            print( "At leaf, predicting %s" % tree['prediction'])
        return tree['prediction'] 
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate: 
            print ("Split on %s = %s" % (tree['splitting_feature'], split_feature_value))
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'], x, annotate)

16. Now, let's consider the first example of the validation set and see what the my_decision_tree_new model predicts for this data point. Your code should be analogous to

In [46]:
print(validation_data.iloc[0])
print('Predicted class: %s ' % classify(my_decision_tree_new, validation_data.iloc[0]))

safe_loans                -1.0
grade_A                    0.0
grade_B                    0.0
grade_C                    0.0
grade_D                    1.0
grade_E                    0.0
grade_F                    0.0
grade_G                    0.0
term_ 36 months            0.0
term_ 60 months            1.0
home_ownership_MORTGAGE    0.0
home_ownership_OTHER       0.0
home_ownership_OWN         0.0
home_ownership_RENT        1.0
emp_length_10+years        0.0
emp_length_1year           0.0
emp_length_2years          1.0
emp_length_3years          0.0
emp_length_4years          0.0
emp_length_5years          0.0
emp_length_6years          0.0
emp_length_7years          0.0
emp_length_8years          0.0
emp_length_9years          0.0
emp_length_<1year          0.0
emp_length_nan             0.0
Name: 0, dtype: float64
Predicted class: -1 


17. Let's add some annotations to our prediction to see what the prediction path was that lead to this predicted class:

In [47]:
classify(my_decision_tree_new, validation_data.iloc[0], annotate=True)

Split on term_ 36 months = 0.0
Split on grade_A = 0.0
At leaf, predicting -1


-1

18. Let's now recall the prediction path for the decision tree learned in the previous assignment, which we recreated here as my_decision_tree_old.

In [48]:
classify(my_decision_tree_old, validation_data.iloc[0], annotate = True)

Split on term_ 36 months = 0.0
Split on grade_A = 0.0
Split on grade_B = 0.0
Split on grade_C = 0.0
Split on grade_D = 1.0
Split on grade_E = 0.0
At leaf, predicting -1


-1

3. **Quiz question:** For my_decision_tree_new trained with 
- max_depth = 6, 
- min_node_size = 100, 
- min_error_reduction=0.0, 
is the prediction path for validation_set[0]  **shorter**, **longer**, or the **same as** for my_decision_tree_old that ignored the early stopping conditions 2 and 3?

**Ans** Shorter

4. **Quiz question:** For my_decision_tree_new trained with 
- max_depth = 6, 
- min_node_size = 100, 
- min_error_reduction=0.0

is the prediction path for any point always shorter, always longer, always the same, shorter or the same, or longer or the same as for my_decision_tree_old that ignored the early stopping conditions 2 and 3?

**Ans** shorter or the same

5. **Quiz question:** For a tree trained on any dataset using 
- max_depth = 6, 
- min_node_size = 100, 
- min_error_reduction=0.0, 

what is the maximum number of splits encountered while making a single prediction?

**Ans** = max_depth = 6

## Evaluating the model

19. Now let us evaluate the model that we have trained. You implemented this evaluation in the function evaluate_classification_error from the previous assignment. Use your code from the previous assignment. It should be analogous to

we will write a function to evaluate a decision tree by computing the classification error of the tree on the given dataset. Write a function called evaluate_classification_error that takes in as input:

    1. tree (as described above)
    2. data (a data frame of data points)

In [49]:
def evaluate_classification_error(tree, data, target):
    # Apply the classify(tree, x) to each row in your data
    prediction = data.apply(lambda x: classify(tree, x), axis = 1)
    
    # Once you've made the predictions, calculate the classification error and return it
    ## YOUR CODE HERE
    error = (prediction != data[target]).sum()
    error = error/len(data)
    return error

20. Now, let's use this function to evaluate the classification error of my_decision_tree_new on the validation_set. Your code should be analogous to

In [50]:
error_with_new_tree = evaluate_classification_error(my_decision_tree_new, validation_data, "safe_loans")
print(error_with_new_tree )

0.38367083153813014


21. Now, evaluate the validation error using my_decision_tree_old.

In [51]:
error_with_old_tree  = evaluate_classification_error(my_decision_tree_old, validation_data, "safe_loans")
print(error_with_old_tree )

0.3837785437311504


In [52]:
error_with_new_tree - error_with_old_tree 

-0.00010771219302024848

6. **Quiz question:** Is the validation error of the new decision tree (using early stopping conditions 2 and 3) lower than, higher than, or the same as that of the old decision tree from the previous assignment?

**Ans** lower

## Exploring the effect of max_depth

We will compare three models trained with different values of the stopping criterion. We intentionally picked models at the extreme ends (**too small**, **just right**, and **too large**)

22. Train three models with these parameters:

* model_1: max_depth = 2 (too small)
* model_2: max_depth = 6 (just right)
* model_3: max_depth = 14 (may be too large)

For each of these three, set min_node_size = 0 and min_error_reduction = -1. Make sure to call the models model_1, model_2, and model_3.

In [53]:
model_1 = decision_tree_create(train_data, new_features, 'safe_loans', max_depth = 2, min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 1 (28001 data points).
Split on feature grade_D. (23300, 4701)
--------------------------------------------------------------------
Subtree, depth = 2 (23300 data points).
Early stopping condition 1 reached. Reached maximum depth.
-----------------------------------------------

In [54]:
model_2 = decision_tree_create(train_data, new_features, 'safe_loans', max_depth = 6, min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
E

Split on feature grade_G. (20638, 96)
--------------------------------------------------------------------
Subtree, depth = 6 (20638 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 6 (96 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 5 (932 data points).
Split on feature grade_A. (702, 230)
--------------------------------------------------------------------
Subtree, depth = 6 (702 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 6 (230 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 4 (358 data points).
Split on feature emp_length_8years. (347, 11)


In [55]:
model_3 = decision_tree_create(train_data, new_features, 'safe_loans', max_depth = 14, min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
S

Split on feature grade_F. (2133, 0)
--------------------------------------------------------------------
Subtree, depth = 7 (2133 data points).
Split on feature grade_G. (2133, 0)
--------------------------------------------------------------------
Subtree, depth = 8 (2133 data points).
Split on feature term_ 60 months. (0, 2133)
--------------------------------------------------------------------
Subtree, depth = 9 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
--------------------------------------------------------------------
Subtree, depth = 9 (2133 data points).
Split on feature home_ownership_MORTGAGE. (1045, 1088)
--------------------------------------------------------------------
Subtree, depth = 10 (1045 data points).
Split on feature home_ownership_OTHER. (1044, 1)
--------------------------------------------------------------------
Subtree, depth = 11 (1044 data points).
Split on feature home_ownership_OWN. (879, 165)
-----------

Split on feature grade_D. (2190, 0)
--------------------------------------------------------------------
Subtree, depth = 5 (2190 data points).
Split on feature grade_E. (2190, 0)
--------------------------------------------------------------------
Subtree, depth = 6 (2190 data points).
Split on feature grade_F. (2190, 0)
--------------------------------------------------------------------
Subtree, depth = 7 (2190 data points).
Split on feature grade_G. (2190, 0)
--------------------------------------------------------------------
Subtree, depth = 8 (2190 data points).
Split on feature term_ 60 months. (0, 2190)
--------------------------------------------------------------------
Subtree, depth = 9 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
--------------------------------------------------------------------
Subtree, depth = 9 (2190 data points).
Split on feature home_ownership_MORTGAGE. (803, 1387)
---------------------------------------

Split on feature term_ 60 months. (0, 969)
--------------------------------------------------------------------
Subtree, depth = 10 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
--------------------------------------------------------------------
Subtree, depth = 10 (969 data points).
Split on feature home_ownership_MORTGAGE. (367, 602)
--------------------------------------------------------------------
Subtree, depth = 11 (367 data points).
Split on feature home_ownership_OTHER. (367, 0)
--------------------------------------------------------------------
Subtree, depth = 12 (367 data points).
Split on feature home_ownership_OWN. (291, 76)
--------------------------------------------------------------------
Subtree, depth = 13 (291 data points).
Split on feature home_ownership_RENT. (0, 291)
--------------------------------------------------------------------
Subtree, depth = 14 (0 data points).
Stopping condition 1 reached. All data point

Split on feature grade_E. (45, 0)
--------------------------------------------------------------------
Subtree, depth = 8 (45 data points).
Split on feature grade_F. (45, 0)
--------------------------------------------------------------------
Subtree, depth = 9 (45 data points).
Split on feature grade_G. (45, 0)
--------------------------------------------------------------------
Subtree, depth = 10 (45 data points).
Split on feature term_ 60 months. (0, 45)
--------------------------------------------------------------------
Subtree, depth = 11 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
--------------------------------------------------------------------
Subtree, depth = 11 (45 data points).
Split on feature home_ownership_OTHER. (45, 0)
--------------------------------------------------------------------
Subtree, depth = 12 (45 data points).
Split on feature home_ownership_OWN. (45, 0)
---------------------------------------------------

...

Split on feature home_ownership_OWN. (47, 0)
--------------------------------------------------------------------
Subtree, depth = 14 (47 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 14 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
--------------------------------------------------------------------
Subtree, depth = 13 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
--------------------------------------------------------------------
Subtree, depth = 12 (5 data points).
Split on feature home_ownership_OTHER. (5, 0)
--------------------------------------------------------------------
Subtree, depth = 13 (5 data points).
Split on feature home_ownership_OWN. (5, 0)
--------------------------------------------------------------------
Subtree, depth = 14 (5 data points).
Early stopping condit

Split on feature grade_B. (230, 0)
--------------------------------------------------------------------
Subtree, depth = 7 (230 data points).
Split on feature grade_C. (230, 0)
--------------------------------------------------------------------
Subtree, depth = 8 (230 data points).
Split on feature grade_G. (230, 0)
--------------------------------------------------------------------
Subtree, depth = 9 (230 data points).
Split on feature term_ 60 months. (230, 0)
--------------------------------------------------------------------
Subtree, depth = 10 (230 data points).
Split on feature home_ownership_MORTGAGE. (119, 111)
--------------------------------------------------------------------
Subtree, depth = 11 (119 data points).
Split on feature home_ownership_OTHER. (119, 0)
--------------------------------------------------------------------
Subtree, depth = 12 (119 data points).
Split on feature home_ownership_OWN. (71, 48)
------------------------------------------------------------

Stopping condition 1 reached. All data points have the same target value.
--------------------------------------------------------------------
Subtree, depth = 6 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
--------------------------------------------------------------------
Subtree, depth = 5 (11 data points).
Split on feature home_ownership_OWN. (9, 2)
--------------------------------------------------------------------
Subtree, depth = 6 (9 data points).
Split on feature grade_A. (9, 0)
--------------------------------------------------------------------
Subtree, depth = 7 (9 data points).
Split on feature grade_B. (9, 0)
--------------------------------------------------------------------
Subtree, depth = 8 (9 data points).
Split on feature grade_C. (9, 0)
--------------------------------------------------------------------
Subtree, depth = 9 (9 data points).
Split on feature grade_G. (9, 0)
---------------------------------------------

Split on feature home_ownership_OTHER. (13, 0)
--------------------------------------------------------------------
Subtree, depth = 12 (13 data points).
Split on feature home_ownership_OWN. (13, 0)
--------------------------------------------------------------------
Subtree, depth = 13 (13 data points).
Split on feature home_ownership_RENT. (13, 0)
--------------------------------------------------------------------
Subtree, depth = 14 (13 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 14 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
--------------------------------------------------------------------
Subtree, depth = 13 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
--------------------------------------------------------------------
Subtree, depth = 12 (0 data points).
Stopping conditi

Split on feature home_ownership_OWN. (122, 0)
--------------------------------------------------------------------
Subtree, depth = 13 (122 data points).
Split on feature home_ownership_RENT. (122, 0)
--------------------------------------------------------------------
Subtree, depth = 14 (122 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 14 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
--------------------------------------------------------------------
Subtree, depth = 13 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
--------------------------------------------------------------------
Subtree, depth = 12 (0 data points).
Stopping condition 1 reached. All data points have the same target value.
--------------------------------------------------------------------
Subtree, depth = 9 (0 d

23. Let us evaluate the models on the train and validation data. Let us start by evaluating the classification error on the training data. Your code should be analogous to:

In [56]:
print ("Training data, classification error (model 1):", evaluate_classification_error(model_1, train_data, 'safe_loans'))
print ("Training data, classification error (model 2):", evaluate_classification_error(model_2, train_data, 'safe_loans'))
print ("Training data, classification error (model 3):", evaluate_classification_error(model_3, train_data, 'safe_loans'))

Training data, classification error (model 1): 0.40003761014399314
Training data, classification error (model 2): 0.38185041908446166
Training data, classification error (model 3): 0.37446271222866967


24. Now evaluate the classification error on the validation data.

In [57]:
print ("validation data, classification error (model 1):", evaluate_classification_error(model_1, validation_data, 'safe_loans'))
print ("validation data, classification error (model 2):", evaluate_classification_error(model_2, validation_data, 'safe_loans'))
print ("validation data, classification error (model 3):", evaluate_classification_error(model_3, validation_data, 'safe_loans'))

validation data, classification error (model 1): 0.3981042654028436
validation data, classification error (model 2): 0.3837785437311504
validation data, classification error (model 3): 0.38000861697544164


7. **Quiz Question:** Which tree has the smallest error on the validation data?

**Ans** model_3

8. **Quiz Question:** Does the tree with the smallest error in the training data also have the smallest error in the validation data?

**Ans** Yes

9. **Quiz Question:** Is it always true that the tree with the lowest classification error on the training set will result in the lowest classification error in the validation set?

**Ans** No

## Measuring the complexity of the tree

complexity(T) = number of leaves in the tree T

25. Here, we provide a function count_leaves that counts the number of leaves in a tree. Using this implementation, compute the number of nodes in model_1, model_2, and model_3. This code is in Python. If you are using another language, make sure your code is analogous to

In [58]:
def count_leaves(tree):
    if tree['is_leaf']:
        return 1
    return count_leaves(tree['left']) + count_leaves(tree['right'])

26. Using the function count_leaves, compute the number of nodes in model_1, model_2, and model_3.

In [59]:
# complexity of model_1
count_leaves(model_1)

4

In [60]:
# complexity of model_2
count_leaves(model_2)

41

In [61]:
# complexity of model_3
count_leaves(model_3)

341

10. **Quiz question:** Which tree has the largest complexity?

**Ans** model 3

11. **Quiz question:** Is it always true that the most complex tree will result in the lowest classification error in the validation_set?

**Ans** No

## Exploring the effect of min_error

We will compare three models trained with different values of the stopping criterion. We intentionally picked models at the extreme ends (negative, just right, and too positive).

27. Train three models with these parameters:

* **model_4:** min_error_reduction = -1 (ignoring this early stopping condition)
* **model_5:** min_error_reduction = 0 (just right)
* **model_6:** min_error_reduction = 5 (too positive)

For each of these three, we set max_depth = 6, and min_node_size = 0. Make sure to call the models model_4, model_5, and model_6.

In [62]:
model_4 = decision_tree_create(train_data, new_features, 'safe_loans',\
                               max_depth = 6, min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
E

Split on feature grade_G. (20638, 96)
--------------------------------------------------------------------
Subtree, depth = 6 (20638 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 6 (96 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 5 (932 data points).
Split on feature grade_A. (702, 230)
--------------------------------------------------------------------
Subtree, depth = 6 (702 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 6 (230 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 4 (358 data points).
Split on feature emp_length_8years. (347, 11)


In [63]:
model_5 = decision_tree_create(train_data, new_features, 'safe_loans', max_depth = 6, min_node_size = 0, min_error_reduction=0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Early stopping condition 3 reached. Minimum error reduction.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Split on feature emp_length_nan. (96, 5)
--------------------------------------------------------------------
Subtree, depth = 3 (96 data points).
Split on feature emp_length_<1year. (85, 11)
--------------------------------------------------------------------
Subtree, depth = 4 (85 data points).
Early stopping condition 3 reached. Minimum error reduction.
--------------------------------------------------------------

In [64]:
model_6 = decision_tree_create(train_data, new_features, 'safe_loans', max_depth = 6, min_node_size = 0, min_error_reduction=5)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Early stopping condition 3 reached. Minimum error reduction.


28. Calculate the accuracy of each model (model_4, model_5, or model_6) on the validation set. Your code should be analogous to

In [65]:
print ("validation data, classification error (model 4):", evaluate_classification_error(model_4, validation_data, 'safe_loans'))
print ("validation data, classification error (model 5):", evaluate_classification_error(model_5, validation_data, 'safe_loans'))
print ("validation data, classification error (model 6):", evaluate_classification_error(model_6, validation_data, 'safe_loans'))

validation data, classification error (model 4): 0.3837785437311504
validation data, classification error (model 5): 0.3837785437311504
validation data, classification error (model 6): 0.503446790176648


29. Using the count_leaves function, compute the number of leaves in each of each models in (model_4, model_5, and model_6).

In [66]:
# complexity of models
print("complexity of model 4: ", count_leaves(model_4))
print("complexity of model 5: ", count_leaves(model_5))
print("complexity of model 6: ", count_leaves(model_6))

complexity of model 4:  41
complexity of model 5:  13
complexity of model 6:  1


12. **Quiz Question:** Using the complexity definition above, which model (model_4, model_5, or model_6) has the largest complexity? Did this match your expectation?

**Ans** model 4

13. **Quiz Question:** model_4 and model_5 have similar classification error on the validation set but model_5 has lower complexity? Should you pick model_5 over model_4?

**Ans** Yes

## Exploring the effect of min_node_size


We will compare three models trained with different values of the stopping criterion. Again, intentionally picked models at the extreme ends (too small, just right, and just right).

30. Train three models with these parameters:

* model_7: min_node_size = 0 (too small)
* model_8: min_node_size = 2000 (just right)
* model_9: min_node_size = 50000 (too large)

For each of these three, we set max_depth = 6, and min_error_reduction = -1. Make sure to call these models model_7, model_8, and model_9.

In [67]:
model_7 = decision_tree_create(train_data, new_features, 'safe_loans',\
                               max_depth = 6, min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
E

Split on feature grade_G. (20638, 96)
--------------------------------------------------------------------
Subtree, depth = 6 (20638 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 6 (96 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 5 (932 data points).
Split on feature grade_A. (702, 230)
--------------------------------------------------------------------
Subtree, depth = 6 (702 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 6 (230 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 4 (358 data points).
Split on feature emp_length_8years. (347, 11)


In [68]:
model_8 = decision_tree_create(train_data, new_features, 'safe_loans',\
                               max_depth = 6, min_node_size = 2000, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
E

In [69]:
model_9 = decision_tree_create(train_data, new_features, 'safe_loans',\
                               max_depth = 6, min_node_size = 50000, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Early stopping condition 2 reached. Reached minimum node size.


31. Calculate the accuracy of each model (model_7, model_8, or model_9) on the validation set.

In [70]:
print ("validation data, classification error (model 7):", evaluate_classification_error(model_7, validation_data, 'safe_loans'))
print ("Tvalidation data, classification error (model 8):", evaluate_classification_error(model_8, validation_data, 'safe_loans'))
print ("validation data, classification error (model 9):", evaluate_classification_error(model_9, validation_data, 'safe_loans'))

validation data, classification error (model 7): 0.3837785437311504
Tvalidation data, classification error (model 8): 0.38453252908229213
validation data, classification error (model 9): 0.503446790176648


32. Using the count_leaves function, compute the number of leaves in each of each models (model_7, model_8, and model_9).

In [71]:
# complexity of models
print("complexity of model 7: ", count_leaves(model_7))
print("complexity of model 8: ", count_leaves(model_8))
print("complexity of model 9: ", count_leaves(model_9))

complexity of model 7:  41
complexity of model 8:  19
complexity of model 9:  1


14. **Quiz Question:** Using the results obtained in this section, which model (model_7, model_8, or model_9) would you choose to use?

**Ans** model 8 (have lesser complexity than model 7)

In [72]:
s1 = ['a', 'b', np.nan]

In [73]:
pd.get_dummies(s1, dummy_na=True, prefix = True)

,True_a,True_b,True_nan
0,1,0,0
1,0,1,0
2,0,0,1


In [74]:
total_data['grade']

KeyError: 'grade'